# Photomath assignment

The assignment was to provide an implementation that can read "very very pretty" handwritten math expressions and calculate the result.
I haven't been able to achieve the best results, as my final implementation still has trouble with some symbols (brackets and minus symbol). However, considering this was my first time using OpenCV and one of the first encounters with CNNs, it was a fun learning experience.

In [56]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2 as cv

In [57]:
NUM_CLASSES = 16
MAX_EPOCHS = 10
PICKLE_FILE = 'dataset/dataset_large.pickle'
MAP_SYMBOLS = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
               '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
               '+': 10, '-': 11, 
              'times': 12, 'div': 13, '(': 14, ')': 15}
VEC_SYMBOLS = np.vectorize(MAP_SYMBOLS.get)

First part of the assignment was to detect each character and find its bounding box. This was done with OpenCV in the python file <code>detector.py</code>
In order to classify the cropped characters, we need to train a CNN, and for that we need a dataset.
After trying out many datasets that I could find online, I've settled on the following Kaggle dataset (https://www.kaggle.com/xainano/handwrittenmathsymbols). 
I've had some trouble detecting '/' as the division operator (it would often get mixed up with the digit 1), so I've decided to use '÷' for the divison operator. Since there was a disbalance in the number of examples for some symbols, I've selected approx. 4000 images for each symbol and preprocessed the images to 'MNIST format'.

## Loading the data, splitting into training and testing sets and reshaping

In [116]:
with open(PICKLE_FILE, 'rb') as f:
    data = pickle.load(f)

X = np.array(data['img'])
y = VEC_SYMBOLS(np.array(data['label']))
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

## CNN model

In [118]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape =(28, 28, 1), activation ='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(15, (3, 3), activation ='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation ='relu'))
model.add(Dense(50, activation ='relu'))
model.add(Dense(NUM_CLASSES, activation ='softmax'))
# Compile model
model.compile(loss ='categorical_crossentropy', 
              optimizer ='adam', metrics =['accuracy'])

In [119]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [120]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_accuracy', patience=4, verbose=1, min_delta=0.01)
mc = ModelCheckpoint('bestmodel.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
cb = [es, mc]

In [121]:
history = model.fit(X_train, y_train, epochs=MAX_EPOCHS, callbacks=cb, batch_size=200, shuffle=True, verbose=1, validation_split=0.1)

Epoch 1/10
203/204 [============================>.] - ETA: 0s - loss: 0.9137 - accuracy: 0.7212
Epoch 00001: val_accuracy improved from -inf to 0.90097, saving model to bestmodel_operators.h5
204/204 [==============================] - 40s 187ms/step - loss: 0.9138 - accuracy: 0.7212 - val_loss: 0.3518 - val_accuracy: 0.9010
Epoch 2/10
203/204 [============================>.] - ETA: 0s - loss: 0.3426 - accuracy: 0.8958
Epoch 00002: val_accuracy improved from 0.90097 to 0.93199, saving model to bestmodel_operators.h5
204/204 [==============================] - 37s 184ms/step - loss: 0.3425 - accuracy: 0.8958 - val_loss: 0.2348 - val_accuracy: 0.9320
Epoch 3/10
203/204 [============================>.] - ETA: 0s - loss: 0.2356 - accuracy: 0.9304
Epoch 00003: val_accuracy improved from 0.93199 to 0.95525, saving model to bestmodel_operators.h5
204/204 [==============================] - 34s 168ms/step - loss: 0.2356 - accuracy: 0.9304 - val_loss: 0.1659 - val_accuracy: 0.9553
Epoch 4/10
203/2

In [110]:
model.evaluate(X_operators_test, y_operators_test)

126/126 [==============================] - 2s 14ms/step - loss: 0.0413 - accuracy: 0.9925


[0.04131867364048958, 0.992516815662384]

In [50]:
model.save('bestmodel.h5')